# Search Planet Basemap API
This script searches the basemap API for basemap grids across the circumpolar region.

## API and Package Set-Up

In [1]:
import os
import json
import requests
import urllib.request
import re
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely as shp
from pprint import pprint

In [2]:
# Get Planet API Key
%load_ext dotenv
%dotenv

api_key = os.getenv('PL_BM_API_KEY')

## Search for Basemap

In [3]:
#setup Planet base URL
API_URL = "https://api.planet.com/basemaps/v1/mosaics"

#setup session
session = requests.Session()

#authenticate
session.auth = (api_key, "")

In [4]:
# make get request to access mosaic from basemaps API
res = session.get(API_URL)

#response status code - should be 200 if there were no errors
print(res.status_code)

#print metadata for mosaic
mosaic = res.json()
# print(json.dumps(mosaic, indent=2))


200


In [16]:
#make get request to access mosaic from basemaps API
res = session.get("https://api.planet.com/basemaps/v1/series")

#response status code
print(res.status_code)

#print names of mosaic options
series = res.json()
names = [item['name'] for item in series['series']]
pprint(names)

# print names of global Q3 (July-Sept) mosaics
mosaic_link = [
    mosaic['_links']['mosaics'] for mosaic in series['series'] if mosaic['name'] == 'Global Quarterly'
][0]
pprint(mosaic_link)
mosaics = session.get(mosaic_link).json()
mosaic_names = [mosaic['name'] for mosaic in mosaics['mosaics'] if re.match('^.*q3.*$', mosaic['name'])]
pprint(mosaic_names)
years = [re.search('\d{4}', mosaic)[0] for mosaic in mosaic_names]
years

200
['Global Monthly',
 'Global Quarterly',
 'PS analytic_sr biweekly subscription',
 'PS analytic_sr daily subscription',
 'PS analytic_sr monthly subscription',
 'PS analytic_sr quarterly subscription',
 'PS analytic_sr twice_monthly subscription',
 'PS analytic_sr weekly subscription',
 'PS normalized_analytic biweekly subscription',
 'PS normalized_analytic daily subscription',
 'PS normalized_analytic monthly L16 subscription',
 'PS normalized_analytic monthly subscription',
 'PS normalized_analytic quarterly subscription',
 'PS normalized_analytic twice_monthly subscription',
 'PS normalized_analytic weekly subscription',
 'PS sen2_normalized_analytic monthly subscription',
 'PS sen2_normalized_analytic quarterly subscription',
 'PS sen2_normalized_analytic_8b_sr monthly subscription',
 'PS visual biweekly subscription',
 'PS visual daily subscription',
 'PS visual twice_monthly subscription',
 'PS visual weekly subscription']
'https://api.planet.com/basemaps/v1/series/060c47b7-5

['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']

In [17]:
# set bounding box
string_bbox = '-180,40,180,85'

#search for mosaic quad using AOI
search_parameters = {
    'bbox': string_bbox,
    'minimal': True
}

In [31]:
item_name = []
item_id = []
link = []
percent_covered = []
geom = []
for mosaic_name in mosaic_names:

    #set params for search using name of mosaic
    parameters = {
        "name__is": mosaic_name
    }

    #make get request to access mosaic from basemaps API
    res = session.get(API_URL, params = parameters)

    #response status code
    print(res.status_code)

    #print metadata for mosaic
    mosaic = res.json()
    print(json.dumps(mosaic, indent=2))

    #get id
    mosaic_id = mosaic['mosaics'][0]['id']

    #get bbox for entire mosaic
    mosaic_bbox = mosaic['mosaics'][0]['bbox']

    print(mosaic_id)
    print(mosaic_bbox)

    #accessing quads using metadata from mosaic
    quads_url = "{}/{}/quads".format(API_URL, mosaic_id)

    #send request
    res = session.get(quads_url, params=search_parameters, stream=True)

    quads = res.json()
    pprint(quads)

    item_name = item_name + [mosaic_name]*len(quads['items'])
    item_id = item_id + [item['id'] for item in quads['items']]
    link = link + [item['_links']['download'] for item in quads['items']]
    percent_covered = percent_covered + [item['percent_covered'] for item in quads['items']]
    geom = geom + [
                shp.geometry.box(item['bbox'][0], item['bbox'][1], item['bbox'][2], item['bbox'][3]) 
                for item 
                in quads['items']
            ]
    # print(item_id)

    try:
        next_link = quads['_links']['_next']
        # print(next_link)
    except:
        next_link = None

    while next_link:
        res = session.get(next_link, stream=True)
        quads = res.json()

        item_name = item_name + [mosaic_name]*len(quads['items'])
        item_id = item_id + [item['id'] for item in quads['items']]
        link = link + [item['_links']['download'] for item in quads['items']]
        percent_covered = percent_covered + [item['percent_covered'] for item in quads['items']]
        geom = geom + [
            shp.geometry.box(item['bbox'][0], item['bbox'][1], item['bbox'][2], item['bbox'][3]) 
            for item 
            in quads['items']
        ]

        try:
            next_link = quads['_links']['_next']
            # print(next_link)
        except:
            next_link = None

200
{
  "_links": {
    "_self": "https://api.planet.com/basemaps/v1/mosaics?api_key=PLAKd5b56c4d31f8444387b2eebad24e1c02"
  },
  "mosaics": [
    {
      "_links": {
        "_self": "https://api.planet.com/basemaps/v1/mosaics/e95e2ad5-9689-4ef3-94b7-fbe1310c8785?api_key=PLAKd5b56c4d31f8444387b2eebad24e1c02",
        "quads": "https://api.planet.com/basemaps/v1/mosaics/e95e2ad5-9689-4ef3-94b7-fbe1310c8785/quads?api_key=PLAKd5b56c4d31f8444387b2eebad24e1c02&bbox={lx},{ly},{ux},{uy}",
        "tiles": "https://tiles.planet.com/basemaps/v1/planet-tiles/global_quarterly_2016q3_mosaic/gmap/{z}/{x}/{y}.png?api_key=PLAKd5b56c4d31f8444387b2eebad24e1c02"
      },
      "bbox": [
        -180,
        -56,
        180,
        76
      ],
      "coordinate_system": "EPSG:3857",
      "datatype": "byte",
      "first_acquired": "2016-07-01T00:00:00.000Z",
      "grid": {
        "quad_size": 2048,
        "resolution": 4.777314267823516
      },
      "id": "e95e2ad5-9689-4ef3-94b7-fbe1310c8785",

In [28]:
print(mosaic_name)
print(next_link)
print(res)
res.reason

global_quarterly_2016q3_mosaic
https://api.planet.com/basemaps/v1/mosaics/e95e2ad5-9689-4ef3-94b7-fbe1310c8785/quads?_page=865-3075&_page_size=50&api_key=PLAKd5b56c4d31f8444387b2eebad24e1c02&bbox=-180%2C40%2C180%2C85&minimal=true
<Response [502]>


'Bad Gateway'

In [29]:
problem_link = next_link
print(problem_link)

https://api.planet.com/basemaps/v1/mosaics/e95e2ad5-9689-4ef3-94b7-fbe1310c8785/quads?_page=865-3075&_page_size=50&api_key=PLAKd5b56c4d31f8444387b2eebad24e1c02&bbox=-180%2C40%2C180%2C85&minimal=true


In [30]:
test_response = session.get(next_link, stream=True)
info = test_response.json()
info

{'_links': {'_next': 'https://api.planet.com/basemaps/v1/mosaics/e95e2ad5-9689-4ef3-94b7-fbe1310c8785/quads?_page=915-3075&_page_size=50&api_key=PLAKd5b56c4d31f8444387b2eebad24e1c02&bbox=-180%2C40%2C180%2C85&minimal=true',
  '_self': 'https://api.planet.com/basemaps/v1/mosaics/e95e2ad5-9689-4ef3-94b7-fbe1310c8785/quads?_page=865-3075&_page_size=50&api_key=PLAKd5b56c4d31f8444387b2eebad24e1c02&bbox=-180%2C40%2C180%2C85'},
 'items': [{'_links': {'download': 'https://api.planet.com/basemaps/v1/mosaics/e95e2ad5-9689-4ef3-94b7-fbe1310c8785/quads/866-3075/full?api_key=PLAKd5b56c4d31f8444387b2eebad24e1c02'},
   'bbox': [-103.88671875000001,
    66.6181218846659,
    -103.79882812500001,
    66.65297740055279],
   'id': '866-3075',
   'percent_covered': None},
  {'_links': {'download': 'https://api.planet.com/basemaps/v1/mosaics/e95e2ad5-9689-4ef3-94b7-fbe1310c8785/quads/867-3075/full?api_key=PLAKd5b56c4d31f8444387b2eebad24e1c02'},
   'bbox': [-103.79882812500001,
    66.6181218846659,
    -103

In [32]:
circumpolar_grids = gpd.GeoDataFrame(
    data = pd.DataFrame({
        'name': item_name,
        'id': item_id,
        'link': link,
        'percent_covered': percent_covered,
    }),
    geometry = gpd.GeoSeries(
        data = geom,
        crs = 'EPSG:4326'
    )
)

circumpolar_grids

,name,id,link,percent_covered,geometry
0,global_quarterly_2016q3_mosaic,661-3415,https://api.planet.com/basemaps/v1/mosaics/e95...,None,"POLYGON ((-121.81641 75.99484, -121.81641 76.0..."
1,global_quarterly_2016q3_mosaic,662-3415,https://api.planet.com/basemaps/v1/mosaics/e95...,None,"POLYGON ((-121.72852 75.99484, -121.72852 76.0..."
2,global_quarterly_2016q3_mosaic,663-3415,https://api.planet.com/basemaps/v1/mosaics/e95...,None,"POLYGON ((-121.64063 75.99484, -121.64063 76.0..."
3,global_quarterly_2016q3_mosaic,664-3415,https://api.planet.com/basemaps/v1/mosaics/e95...,None,"POLYGON ((-121.55273 75.99484, -121.55273 76.0..."
4,global_quarterly_2016q3_mosaic,665-3415,https://api.planet.com/basemaps/v1/mosaics/e95...,None,"POLYGON ((-121.46484 75.99484, -121.46484 76.0..."
...,...,...,...,...,...
5902789,global_quarterly_2023q3_mosaic,1828-1272,https://api.planet.com/basemaps/v1/mosaics/a8c...,None,"POLYGON ((141.50391 39.90974, 141.50391 40.044..."
5902790,global_quarterly_2023q3_mosaic,1829-1272,https://api.planet.com/basemaps/v1/mosaics/a8c...,None,"POLYGON ((141.67969 39.90974, 141.67969 40.044..."
5902791,global_quarterly_2023q3_mosaic,1830-1272,https://api.planet.com/basemaps/v1/mosaics/a8c...,None,"POLYGON ((141.85547 39.90974, 141.85547 40.044..."
5902792,global_quarterly_2023q3_mosaic,1831-1272,https://api.planet.com/basemaps/v1/mosaics/a8c...,None,"POLYGON ((142.03125 39.90974, 142.03125 40.044..."


In [33]:
circumpolar_grids.to_file('../data/circumpolar_global_basemap_grids.geojson')